<a href="https://colab.research.google.com/github/1426922668/UHI/blob/main/uhii_code_v05_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio matplotlib scipy scikit-image tqdm
!pip install rasterio
!pip install plotly
!pip install -U kaleido
!pip install -U plotly kaleido pillow

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label
from skimage.measure import regionprops
from skimage.morphology import dilation, square
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
from PIL import Image  # 用于图像格式转换

# config
dem_diff_limit = 100 # 混杂因素中高程差阈值
heat_island_intensity_min_threshold = 0.5 # 小于该阈值热岛观测盘定为不显著
lst_path = 'FY3D_2024.tif'

建成区连通区域（斑块）数量、面积统计

In [ ]:
# 加载数据
with rasterio.open('landcover.tif') as src:
    landcover = src.read(1)  # 读取第一个波段
    meta = src.meta

# 将值为13的像素标记为1，其他为0
binary = (landcover == 13).astype(int)

# 使用scipy的label函数标记连通区域
labeled_array, num_features = label(binary)

# 计算每个斑块的面积
area = np.bincount(labeled_array.ravel())[1:]  # 忽略背景的面积计数

area_min = 1000

# 过滤掉面积小于area_min的斑块
filtered_area = area[area >= area_min]

print(f"大于{area_min}像素的建成区斑块数量: {len(filtered_area)}")

# # 绘制面积-数量统计图
# plt.figure(figsize=(10, 6))
# plt.hist(filtered_area, bins=30, color='blue', alpha=0.7)
# plt.title('建成区斑块面积（大于等于5像素）-建成区数量统计图')
# plt.xlabel('斑块面积（像素数）')
# plt.ylabel('数量')
# plt.grid(True)
# plt.show()


大于1000像素的建成区斑块数量: 6


删除面积过小（当前数据分辨率不适用分级的部分）的建成区斑块并对剩余斑块的索引重新排序

In [ ]:
# 找到大于area_min的斑块索引
valid_indices = np.where(area > area_min)[0] + 1  # 加1因为面积数组的索引从1开始

# 获取有效斑块的面积并排序（从大到小）
sorted_indices = valid_indices[np.argsort(area[valid_indices - 1])[::-1]]  # area索引需减1调整

# 创建一个新数组用于存储重新标记的斑块
relabelled_array = np.zeros_like(labeled_array)

# 根据面积从大到小给斑块重新标记
for rank, index in enumerate(sorted_indices, start=1):
    relabelled_array[labeled_array == index] = rank

# 保存重新标记的斑块数据
with rasterio.open('relabelled_array.tif', 'w', driver='GTiff', height=relabelled_array.shape[0],
                   width=relabelled_array.shape[1], count=1, dtype=relabelled_array.dtype,
                   crs=src.crs, transform=src.transform) as dst:
    dst.write(relabelled_array, 1)

print("重标记后的斑块数据已保存为 'relabelled_array.tif'.")

重标记后的斑块数据已保存为 'relabelled_array.tif'.


根据城建斑块等面积建立热岛分级所需郊区缓冲区并划分热岛

In [ ]:
# 加载DEM数据以除去缓冲区中可能影响热岛分级的过高过低海拔区域
with rasterio.open('dem.tif') as src:
    dem = src.read(1)

# 加载地表温度数据
with rasterio.open(lst_path) as src:
    temperature = src.read(1)
    temp_meta = src.meta

heat_island_grades = np.zeros_like(temperature, dtype=np.uint8)  # 默认0级
heat_island_intensity_img = np.zeros_like(temperature, dtype=np.float32) # 默认热岛强度值为0
buffer_temp_img = np.zeros_like(temperature, dtype=np.float32)

for i in np.unique(relabelled_array)[1:]:
  # i=4 #用于测试
  built_up_patch_mask = (relabelled_array == i)

  # 计算建成区斑块的DEM均值
  patch_dem_value = np.mean(dem[built_up_patch_mask])

  # 计算建成区斑块面积
  patch_area = np.sum(built_up_patch_mask)

  # 生成缓冲区
  buffer_mask = np.copy(built_up_patch_mask)
  buffer_area = np.sum(buffer_mask) - patch_area
  buffer_mask = dilation(built_up_patch_mask)
  best_buffer_mask = buffer_mask.copy()
  min_diff = abs(buffer_area - patch_area)

  # 扩张直到达到或超过斑块面积，同时尝试找到最接近的面积
  # k=0 #用于测试
  while buffer_area < patch_area:
    # k+=1
    buffer_mask = dilation(buffer_mask, square(3))
    buffer_mask[(landcover == 17) | (landcover == 15) | (landcover == 13) | (landcover == 0)] = 0
    buffer_mask[(dem > patch_dem_value + dem_diff_limit) | (dem < patch_dem_value - dem_diff_limit)] = 0
    buffer_area = np.sum(buffer_mask)

    # 检查面积差异
    current_diff = abs(buffer_area - patch_area)
    if current_diff < min_diff:
        min_diff = current_diff
        best_buffer_mask = buffer_mask.copy()

    # print(i , buffer_area , patch_area , np.sum(best_buffer_mask)) #用于测试

    # if k == 200:
    #   break

  # 创建缓冲区栅格
  buffer_raster = best_buffer_mask.astype(np.uint8)

  # 保存缓冲区栅格文件
  meta.update(dtype=rasterio.uint8)
  with rasterio.open('buffer_area_corrected_dem.tif', 'w', **meta) as dst:
      dst.write(buffer_raster, 1)

  print("去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.")

  # 计算建成区斑块的平均温度，排除0值
  built_up_valid_temps = temperature[(built_up_patch_mask) & (temperature != 0)]
  if built_up_valid_temps.size > 0:  # 确保数组不为空
      built_up_mean_temp = built_up_valid_temps.mean()
  else:
      built_up_mean_temp = np.nan  # 如果筛选后没有有效数据，可设为NaN或其他适当值

  # 计算缓冲区的平均温度，排除0值
  buffer_valid_temps = temperature[(best_buffer_mask == 1) & (temperature != 0)]
  if buffer_valid_temps.size > 0:
      buffer_mean_temp = buffer_valid_temps.mean()
  else:
      buffer_mean_temp = np.nan  # 同上，处理空数组情况

  # 热岛强度等级划分
  # 计算热岛强度
  heat_island_intensity = built_up_mean_temp - buffer_mean_temp

  # 未观测到显著热岛强度情况
  if heat_island_intensity < heat_island_intensity_min_threshold:
    print(f"热岛强度值为{heat_island_intensity}，未观测到显著热岛强度")
  else:
    print(f"热岛强度值: {heat_island_intensity}")

  # 定义温度区间
  level_1_upper = buffer_mean_temp - 1.5 * heat_island_intensity
  level_2_upper = buffer_mean_temp - 0.5 * heat_island_intensity
  level_3_upper = buffer_mean_temp + 0.5 * heat_island_intensity
  level_4_upper = buffer_mean_temp + 1.5 * heat_island_intensity

  # # 定义温度区间
  # bj_standard_intensity = 1
  # level_1_upper = buffer_mean_temp + bj_standard_intensity
  # level_2_upper = buffer_mean_temp + 2 * bj_standard_intensity
  # level_3_upper = buffer_mean_temp + 3 * bj_standard_intensity
  # level_4_upper = buffer_mean_temp + 4 * bj_standard_intensity

  # 为最大建成区斑块和隶属区域分配热岛等级
  heat_island_grades[built_up_patch_mask & (temperature < level_1_upper)] = 1
  heat_island_grades[built_up_patch_mask & (temperature >= level_1_upper) & (temperature < level_2_upper)] = 2
  heat_island_grades[built_up_patch_mask & (temperature >= level_2_upper) & (temperature < level_3_upper)] = 3
  heat_island_grades[built_up_patch_mask & (temperature >= level_3_upper) & (temperature < level_4_upper)] = 4
  heat_island_grades[built_up_patch_mask & (temperature >= level_4_upper)] = 5

  # 热岛强度图像
  heat_island_intensity_img[built_up_patch_mask] = heat_island_intensity
  # 缓冲区温度图像
  buffer_temp_img[built_up_patch_mask] = buffer_mean_temp

  # break #用于测试

# 去除0值，即Nodata区域
heat_island_grades = np.where(landcover == 0, 0 ,heat_island_grades)

heat_island_intensity_img[built_up_patch_mask] = heat_island_intensity

# 保存热岛强度分级的tif文件
temp_meta.update(dtype=rasterio.uint8)
with rasterio.open('heat_island_grading.tif', 'w', **temp_meta) as dst:
    dst.write(heat_island_grades, 1)

print("热岛分级tif文件 'heat_island_grading.tif' 已保存.")

# 保存热岛强度分级的tif文件
temp_meta.update(dtype=rasterio.float32)
with rasterio.open('heat_island_intensity_img.tif', 'w', **temp_meta) as dst:
    dst.write(heat_island_intensity_img, 1)

print("热岛强度tif文件 'heat_island_intensity_img.tif' 已保存.")

# 保存缓冲区温度的tif文件
temp_meta.update(dtype=rasterio.float32)
with rasterio.open('buffer_temp_img.tif', 'w', **temp_meta) as dst:
    dst.write(buffer_temp_img, 1)

print("缓冲区温度tif文件 'buffer_temp_img.tif' 已保存.")

去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 1.0147705078125
去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 2.248077392578125
去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 0.800628662109375
去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 1.162200927734375
去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 1.102081298828125
去除混杂因素后的缓冲区栅格文件 'buffer_area_corrected_dem.tif' 已保存.
热岛强度值: 1.7838134765625
热岛分级tif文件 'heat_island_grading.tif' 已保存.
热岛强度tif文件 'heat_island_intensity_img.tif' 已保存.
缓冲区温度tif文件 'buffer_temp_img.tif' 已保存.


In [ ]:
import numpy as np
import rasterio
from scipy.spatial.distance import cdist
from tqdm import tqdm
from scipy.ndimage import binary_erosion

# 加载热岛强度图像
with rasterio.open('heat_island_intensity_img.tif') as src:
    heat_island_intensity_img = src.read(1)
    meta = src.meta

# 加载重新标记的斑块数据
with rasterio.open('relabelled_array.tif') as src:
    relabelled_array = src.read(1)

# 计算各建成区斑块的边缘
patch_edges = {}
for i in np.unique(relabelled_array)[1:]:  # 跳过背景0
    mask = (relabelled_array == i)
    eroded_mask = binary_erosion(mask)
    edge_mask = mask & ~eroded_mask
    y_indices, x_indices = np.where(edge_mask)
    # 采用简化的抽样策略，每50个点抽样一次
    sampled_indices = np.arange(0, len(y_indices), 50)
    patch_edges[i] = np.column_stack((y_indices[sampled_indices], x_indices[sampled_indices]))

print(1)

# 找到缺失数据的坐标
missing_coords = np.column_stack(np.where(heat_island_intensity_img == 0))

# 为每个缺失点计算加权插值
interpolated_values_heat_island_intensity = np.zeros_like(heat_island_intensity_img, dtype=np.float32)
interpolated_values_buffer_mean_temp = np.zeros_like(buffer_temp_img, dtype=np.float32)

for y, x in tqdm(missing_coords, desc="Processing missing coordinates"):
    min_dist = np.inf
    total_weight = 0
    weighted_sum_heat_island_intensity = 0
    weighted_sum_buffer_mean_temp = 0
    for i, edges in patch_edges.items():
        if edges.size > 0:
            # 计算到每个抽样的斑块边缘的距离
            dist = np.min(np.sqrt((edges[:, 0] - y) ** 2 + (edges[:, 1] - x) ** 2))
            if dist < min_dist:
                min_dist = dist
            # 为了简化，使用距离的倒数作为权重
            weight = 1 / (dist + 1e-10)  # 避免除零错误
            weighted_sum_heat_island_intensity += weight * heat_island_intensity_img[edges[:, 0][0], edges[:, 1][0]]  # 用边缘点的强度示例
            weighted_sum_buffer_mean_temp += weight * buffer_temp_img[edges[:, 0][0], edges[:, 1][0]]  # 用边缘点的强度示例
            total_weight += weight

    # 计算该点的插值结果
    if total_weight > 0:
        interpolated_values_heat_island_intensity[y, x] = weighted_sum_heat_island_intensity / total_weight
        interpolated_values_buffer_mean_temp[y, x] = weighted_sum_buffer_mean_temp / total_weight

# 将计算出的插值结果填充到原图中的缺失位置
heat_island_intensity_img[missing_coords[:, 0], missing_coords[:, 1]] = interpolated_values_heat_island_intensity[missing_coords[:, 0], missing_coords[:, 1]]
buffer_temp_img[missing_coords[:, 0], missing_coords[:, 1]] = interpolated_values_buffer_mean_temp[missing_coords[:, 0], missing_coords[:, 1]]

# 保存插值后的热岛强度图像
with rasterio.open('heat_island_intensity_img_interpolated.tif', 'w', **meta) as dst:
    dst.write(heat_island_intensity_img, 1)

print("插值后的热岛强度tif文件 'heat_island_intensity_img_interpolated.tif' 已保存.")

# 保存插值后的缓冲区温度图像
with rasterio.open('buffer_mean_temp_img_interpolated.tif', 'w', **meta) as dst:
    dst.write(buffer_temp_img, 1)

print("插值后的缓冲区温度tif文件 'buffer_mean_temp_img_interpolated.tif' 已保存.")


1


Processing missing coordinates: 100%|██████████| 2016743/2016743 [04:42<00:00, 7131.45it/s]


插值后的热岛强度tif文件 'heat_island_intensity_img_interpolated.tif' 已保存.
插值后的缓冲区温度tif文件 'buffer_mean_temp_img_interpolated.tif' 已保存.


In [ ]:
import numpy as np
import rasterio

# 加载插值后的热岛强度和缓冲区均值图像
with rasterio.open('heat_island_intensity_img_interpolated.tif') as src:
    heat_island_intensity_img = src.read(1)

with rasterio.open('buffer_mean_temp_img_interpolated.tif') as src:
    buffer_mean_temp_img = src.read(1)

# 加载地表温度数据
with rasterio.open(lst_path) as src:
    temperature = src.read(1)
    temp_meta = src.meta

# 初始化热岛等级数组
heat_island_grades_interpolated = np.zeros_like(temperature, dtype=np.uint8)

# 计算温度与缓冲区均值的差值
temperature_diff = temperature - buffer_mean_temp_img

# 计算等级条件
valid_mask = (heat_island_intensity_img != 0)

# 定义温度区间
# bj_standard_intensity = 1
# level_1_upper_interp = temperature_diff - bj_standard_intensity
# level_2_upper_interp = temperature_diff - 2 * bj_standard_intensity
# level_3_upper_interp = temperature_diff - 3 * bj_standard_intensity
# level_4_upper_interp = temperature_diff - 4 * bj_standard_intensity

level_1_upper_interp = temperature_diff + heat_island_intensity_img * 1.5
level_2_upper_interp = temperature_diff + heat_island_intensity_img * 0.5
level_3_upper_interp = temperature_diff - heat_island_intensity_img * 0.5
level_4_upper_interp = temperature_diff - heat_island_intensity_img * 1.5

# 计算热岛等级
heat_island_grades_interpolated[valid_mask & (level_4_upper_interp >= 0)] = 5
heat_island_grades_interpolated[valid_mask & (level_3_upper_interp >= 0) & (level_4_upper_interp < 0)] = 4
heat_island_grades_interpolated[valid_mask & (level_2_upper_interp >= 0) & (level_3_upper_interp < 0)] = 3
heat_island_grades_interpolated[valid_mask & (level_1_upper_interp >= 0) & (level_2_upper_interp < 0)] = 2
heat_island_grades_interpolated[valid_mask & (level_1_upper_interp < 0)] = 1

print("Level 5 pixels:", np.sum(valid_mask & (level_4_upper_interp >= 0)))
print("Level 4 pixels:", np.sum(valid_mask & (level_3_upper_interp >= 0) & (level_4_upper_interp < 0)))
print("Level 3 pixels:", np.sum(valid_mask & (level_2_upper_interp >= 0) & (level_3_upper_interp < 0)))
print("Level 2 pixels:", np.sum(valid_mask & (level_1_upper_interp >= 0) & (level_2_upper_interp < 0)))
print("Level 1 pixels:", np.sum(valid_mask & (level_1_upper_interp < 0)))

# 建成区分级覆盖插值分区
heat_island_grades_interpolated = np.where(heat_island_grades, heat_island_grades ,heat_island_grades_interpolated)

# 去除0值，即Nodata区域
heat_island_grades_interpolated = np.where(temperature == 0, 0 ,heat_island_grades_interpolated)

# 保存热岛等级图像
with rasterio.open('heat_island_grades_interpolated.tif', 'w', **src.meta) as dst:
    dst.write(heat_island_grades_interpolated, 1)

print("插值后的热岛等级tif文件 'heat_island_grades_interpolated.tif' 已保存.")

Level 5 pixels: 38596
Level 4 pixels: 99528
Level 3 pixels: 187712
Level 2 pixels: 133948
Level 1 pixels: 1597252
插值后的热岛等级tif文件 'heat_island_grades_interpolated.tif' 已保存.


In [ ]:
# 读取2022年的热岛等级数据
with rasterio.open('2022_heat_island_grades_interpolated.tif') as src_2022:
    heat_island_2022 = src_2022.read(1)

# 读取2023年的热岛等级数据
with rasterio.open('2023_heat_island_grades_interpolated.tif') as src_2023:
    heat_island_2023 = src_2023.read(1)

# 读取2024年的热岛等级数据
with rasterio.open('2024_heat_island_grades_interpolated.tif') as src_2024:
    heat_island_2024 = src_2024.read(1)

# 检查三个数据集的尺寸是否一致
if not (heat_island_2022.shape == heat_island_2023.shape == heat_island_2024.shape):
    raise ValueError("三个年份的数据尺寸不一致，请检查数据。")

# 将值为0的像素（假设为无效数据）转换为np.nan
heat_island_2022 = np.where(heat_island_2022 == 0, np.nan, heat_island_2022)
heat_island_2023 = np.where(heat_island_2023 == 0, np.nan, heat_island_2023)
heat_island_2024 = np.where(heat_island_2024 == 0, np.nan, heat_island_2024)

# 假设热岛等级为1到5级
grades = [5, 4, 3, 2, 1]#[4, 3, 5, 2, 1]

# 创建第一个转换矩阵
transition_matrix_1 = pd.DataFrame(0, index=grades, columns=grades)

# 遍历每个等级，统计转换次数
for grade_from in grades:
    for grade_to in grades:
        count = np.nansum((heat_island_2022 == grade_from) & (heat_island_2023 == grade_to))
        transition_matrix_1.at[grade_from, grade_to] = count

# 创建第二个转换矩阵
transition_matrix_2 = pd.DataFrame(0, index=grades, columns=grades)

# 遍历每个等级，统计转换次数
for grade_from in grades:
    for grade_to in grades:
        count = np.nansum((heat_island_2023 == grade_from) & (heat_island_2024 == grade_to))
        transition_matrix_2.at[grade_from, grade_to] = count

labels = (
    ['2022级别{}'.format(i) for i in grades] +
    ['2023级别{}'.format(i) for i in grades] +
    ['2024级别{}'.format(i) for i in grades]
)


# 为每个节点创建索引
node_indices = {label: idx for idx, label in enumerate(labels)}


# 初始化列表
source_indices = []
target_indices = []
values = []

# 添加第一阶段的链接（2022年到2023年）
for grade_from in grades:
    for grade_to in grades:
        value = transition_matrix_1.at[grade_from, grade_to]
        if value > 0:
            source_label = '2022级别{}'.format(grade_from)
            target_label = '2023级别{}'.format(grade_to)
            source_indices.append(node_indices[source_label])
            target_indices.append(node_indices[target_label])
            values.append(value)

# 添加第二阶段的链接（2023年到2024年）
for grade_from in grades:
    for grade_to in grades:
        value = transition_matrix_2.at[grade_from, grade_to]
        if value > 0:
            source_label = '2023级别{}'.format(grade_from)
            target_label = '2024级别{}'.format(grade_to)
            source_indices.append(node_indices[source_label])
            target_indices.append(node_indices[target_label])
            values.append(value)


# 创建桑基图
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="2022年到2023年再到2024年热岛等级转换桑基图", font_size=10)
fig.show()


# 保存为HTML文件
fig.write_html("heat_island_transition_sankey_3years.html")

# # 保存为PNG图片（需要安装kaleido）
# !pip install -U kaleido
# !pip install -U kaleido
# import plotly.io as pio
# pio.kaleido.scope.default_format = "png"
# pio.kaleido.scope.default_width = 1000
# pio.kaleido.scope.default_height = 600

# fig.write_image("heat_island_transition_sankey_3years.png")


In [ ]:
import rasterio
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# 读取2022年的热岛等级数据
with rasterio.open('2022_heat_island_grades_interpolated.tif') as src_2022:
    heat_island_2022 = src_2022.read(1)

# 读取2023年的热岛等级数据
with rasterio.open('2023_heat_island_grades_interpolated.tif') as src_2023:
    heat_island_2023 = src_2023.read(1)

# 读取2024年的热岛等级数据
with rasterio.open('2024_heat_island_grades_interpolated.tif') as src_2024:
    heat_island_2024 = src_2024.read(1)

# 检查三个数据集的尺寸是否一致
if not (heat_island_2022.shape == heat_island_2023.shape == heat_island_2024.shape):
    raise ValueError("三个年份的数据尺寸不一致，请检查数据。")

# 将值为0的像素（假设为无效数据）转换为np.nan
heat_island_2022 = np.where(heat_island_2022 == 0, np.nan, heat_island_2022)
heat_island_2023 = np.where(heat_island_2023 == 0, np.nan, heat_island_2023)
heat_island_2024 = np.where(heat_island_2024 == 0, np.nan, heat_island_2024)

# 定义热岛等级列表，从高到低
grades = [5, 4, 3, 2, 1]

# 创建第一个转换矩阵
transition_matrix_1 = pd.DataFrame(0, index=grades, columns=grades)

# 遍历每个等级，统计转换次数（2022年到2023年）
for grade_from in grades:
    for grade_to in grades:
        count = np.nansum((heat_island_2022 == grade_from) & (heat_island_2023 == grade_to))
        transition_matrix_1.at[grade_from, grade_to] = count

# 创建第二个转换矩阵
transition_matrix_2 = pd.DataFrame(0, index=grades, columns=grades)

# 遍历每个等级，统计转换次数（2023年到2024年）
for grade_from in grades:
    for grade_to in grades:
        count = np.nansum((heat_island_2023 == grade_from) & (heat_island_2024 == grade_to))
        transition_matrix_2.at[grade_from, grade_to] = count

# 定义节点标签，按照年份和等级，从高到低
labels = (
    ['2022级别{}'.format(i) for i in grades] +
    ['2023级别{}'.format(i) for i in grades] +
    ['2024级别{}'.format(i) for i in grades]
)

# 为每个节点创建索引
node_indices = {label: idx for idx, label in enumerate(labels)}

# 初始化列表
source_indices = []
target_indices = []
values = []

# 添加第一阶段的链接（2022年到2023年）
for grade_from in grades:
    for grade_to in grades:
        value = transition_matrix_1.at[grade_from, grade_to]
        if value > 0:
            source_label = '2022级别{}'.format(grade_from)
            target_label = '2023级别{}'.format(grade_to)
            source_indices.append(node_indices[source_label])
            target_indices.append(node_indices[target_label])
            values.append(value)

# 添加第二阶段的链接（2023年到2024年）
for grade_from in grades:
    for grade_to in grades:
        value = transition_matrix_2.at[grade_from, grade_to]
        if value > 0:
            source_label = '2023级别{}'.format(grade_from)
            target_label = '2024级别{}'.format(grade_to)
            source_indices.append(node_indices[source_label])
            target_indices.append(node_indices[target_label])
            values.append(value)

# 定义颜色映射，将热岛等级映射到对应的颜色
color_map = {
    5: 'rgb(165, 0, 38)',    # 5级热岛颜色
    4: 'rgb(249, 142, 82)',   # 4级热岛颜色
    3: 'rgb(255, 255, 255)',  # 3级热岛颜色
    2: 'rgb(144, 195, 221)',  # 2级热岛颜色
    1: 'rgb(49, 54, 149)'   # 1级热岛颜色
}

# 生成节点颜色列表
node_colors = []

for label in labels:
    # 从标签中提取热岛等级
    grade = int(label[-1])  # 假设等级是标签的最后一个字符
    node_colors.append(color_map[grade])

# 创建桑基图，应用节点颜色
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color=node_colors  # 使用生成的节点颜色列表
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="2022年到2023年再到2024年热岛等级转换桑基图", font_size=10)
fig.show()

# 保存为HTML文件
fig.write_html("heat_island_transition_sankey_3years.html")

# 保存为PNG图片（需要安装kaleido）
import sys
!{sys.executable} -m pip install -U kaleido

fig.write_image("heat_island_transition_sankey_3years.png")


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
y_positions = np.linspace(0, 1, num_grades)
y_positions

array([0.  , 0.25, 0.5 , 0.75, 1.  ])